In [1]:
import torch
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pandas as pd
import torch.nn as nn


/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.15 (you have 1.4.14). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


#### 유사도에 사용할 모델 가져오기

In [2]:
ckpt = torch.load("result/resnet50-256-0.0005-AdamW-O-augv1_09-16_0/epoch=34-step=1645.ckpt")
state_dict = {'.'.join(key.split('.')[2:]): val for key, val in ckpt['state_dict'].items()}

#### 학습된 weight와 동일한 모델 필요

In [3]:
model = timm.create_model('resnet50', pretrained=False,num_classes=500)
model.load_state_dict(state_dict)

<All keys matched successfully>

In [4]:
if hasattr(model, 'fc'):
    model.fc = nn.Identity() 
elif hasattr(model, 'classifier'):
    model.classifier = nn.Identity()

In [5]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act2): ReLU(inplace=True)
      (aa): Identity()
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

In [6]:
common_transforms = [
    A.Resize(224, 224),  # 이미지를 224x224 크기로 리사이즈
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 정규화
    ToTensorV2()  # albumentations에서 제공하는 PyTorch 텐서 변환
]
transform = A.Compose(common_transforms)

In [7]:
from torch.utils.data import Dataset,DataLoader
from typing import Callable, Union, Tuple
import cv2
import os

class CustomDataset(Dataset):
    def __init__(
        self, 
        root_dir, 
        info_df, 
        transform,
        is_inference
    ):
        # 데이터셋의 기본 경로, 이미지 변환 방법, 이미지 경로 및 레이블을 초기화합니다.
        self.root_dir = root_dir  # 이미지 파일들이 저장된 기본 디렉토리
        self.transform = transform  # 이미지에 적용될 변환 처리
        self.is_inference = is_inference # 추론인지 확인
        self.info_df = info_df
        # self.info_df = info_df.iloc[:100] #TODO 
        self.image_paths = self.info_df['image_path'].tolist()  # 이미지 파일 경로 목록
        
        if not self.is_inference:
            self.targets = self.info_df['target'].tolist()  # 각 이미지에 대한 레이블 목록

    def __len__(self) -> int:
        # 데이터셋의 총 이미지 수를 반환합니다.
        return len(self.image_paths)

    def __getitem__(self, index: int) -> Union[Tuple[torch.Tensor, int], torch.Tensor]:
        # 주어진 인덱스에 해당하는 이미지를 로드하고 변환을 적용한 후, 이미지와 레이블을 반환합니다.
        img_path = os.path.join(self.root_dir, self.image_paths[index])  # 이미지 경로 조합
        image = cv2.imread(img_path, cv2.IMREAD_COLOR)  # 이미지를 BGR 컬러 포맷의 numpy array로 읽어옵니다.
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # BGR 포맷을 RGB 포맷으로 변환합니다.
        image = self.transform(image=image)  # 설정된 이미지 변환을 적용합니다.

        if self.is_inference:
            return image
        else:
            target = self.targets[index]  # 해당 이미지의 레이블
            return image['image'], target, index# 변환된 이미지와 레이블을 튜플 형태로 반환합니다. 

In [8]:
info_df = pd.read_csv('./data/train.csv')

In [9]:
dup_dataset = CustomDataset('./data/train',info_df,transform,False)

In [10]:
dup_dataloader = DataLoader(dup_dataset, batch_size=512, num_workers=4, shuffle=False)

In [11]:
model.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act2): ReLU(inplace=True)
      (aa): Identity()
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

#### 이미지 embedding 추출

In [12]:
feats = []
ys = []
idxes = []
with torch.no_grad():
    for batch in dup_dataloader:
        x,y,idx = batch
        x = x.cuda()
        feat = model(x)
        feats.append(feat)
        ys.append(y)
        idxes.append(idx)

feats = torch.concat(feats)
ys = torch.concat(ys)      
idxes = torch.concat(idxes)  

In [13]:
print(ys.shape)
print(idxes.shape)
print(feats.shape)

torch.Size([15021])
torch.Size([15021])
torch.Size([15021, 2048])


In [14]:
import torch.nn.functional as F


In [15]:
feats = F.normalize(feats, p=2, dim=1)


In [16]:
cosine_similarity_matrix = torch.matmul(feats, feats.T)


In [17]:
cosine_similarity_matrix.shape

torch.Size([15021, 15021])

In [51]:
# Upper triangular matrix에서 threshold 이상인 값의 index 찾기
indices = torch.triu(cosine_similarity_matrix, diagonal=1) > 0.97



In [52]:
# 조건을 만족하는 i, j 쌍의 index 추출
pairs = torch.nonzero(indices, as_tuple=False)


In [53]:
pairs.shape

torch.Size([2771, 2])

In [54]:
from tqdm import tqdm

In [55]:
high_similarity_pairs = []
for i,j in tqdm(pairs):
    high_similarity_pairs.append((idxes[i].item(), idxes[j].item(), cosine_similarity_matrix[i, j].item()) )

100%|██████████| 2771/2771 [00:00<00:00, 13024.21it/s]


In [56]:
len(high_similarity_pairs)

2771

In [63]:
high_similarity_pairs.sort(key=lambda x:x[2])

In [57]:
paths = info_df['image_path'].tolist()

In [64]:
high_similarity_pairs[0]

(2911, 6617, 0.9700164198875427)

In [65]:
train_root = './data/train'

#### 결과 이미지 출력

In [66]:
import os
import cv2
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import clear_output

# 데이터 준비 (이 부분은 실제 코드에 맞게 조정해야 합니다)
# miss_df, val_data_dir, class_name_to_idx 등의 데이터와 디렉토리 설정이 필요합니다

def plot_image(value):
    value = int(value)  # 슬라이더 값 추출
    clear_output(wait=True)  # 이전 출력 제거
    fig, ax = plt.subplots(1, 2, figsize=(6, 12))  # 새 그림 생성

    x1,x2,sim = high_similarity_pairs[value]
    # 잘못 예측된 sample (miss_df에서 잘못 예측된 이미지 경로)
    img_path = os.path.join(train_root, paths[x1])
    image = cv2.imread(img_path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    ax[0].imshow(image)
    x1_label = ys[x1].item()
    ax[0].set_title(f'{x1_label},sim:{sim}')
    ax[0].axis('off')
    

    img_path = os.path.join(train_root, paths[x2])
    image = cv2.imread(img_path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    ax[1].imshow(image)
    x2_label = ys[x2].item()
    ax[1].set_title(f'{x2_label}')
    ax[1].axis('off')


    plt.tight_layout()
    plt.show()

# 슬라이더 생성
slider = widgets.IntSlider(min=0, max=len(high_similarity_pairs) - 1, step=1, description='Index')

# 슬라이더와 plot_image 함수 연결
widgets.interact(plot_image, value=slider)


interactive(children=(IntSlider(value=0, description='Index', max=2770), Output()), _dom_classes=('widget-inte…

<function __main__.plot_image(value)>

#### CSV 저장을 위한 dictionary 

In [67]:
path_dict = {}
for idx,(i,j,sim) in enumerate(high_similarity_pairs):
    path_dict[idx] = (paths[i],paths[j],sim,ys[i].item(),ys[j].item())

In [69]:
res_df = pd.DataFrame(path_dict).T

In [70]:
len(res_df[res_df[3]==res_df[4]]) 

2609

In [71]:
len(set(res_df[res_df[3]==res_df[4]][1]))

2008

In [72]:
class_df = res_df[res_df[3]==res_df[4]]

In [73]:
class_df['class'] = class_df[0].apply(lambda x: x.split('/')[0])

/tmp/ipykernel_789012/856211214.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  class_df['class'] = class_df[0].apply(lambda x: x.split('/')[0])


In [74]:
class_df[class_df['class']=='n02110958']

,0,1,2,3,4,class
464,n02110958/sketch_48.JPEG,n02110958/sketch_2.JPEG,0.974354,150,150,n02110958


#### class가 다르지만, 유사한 이미지 저장

In [76]:
tt = res_df[res_df[3]!=res_df[4]]

In [77]:
tt.to_csv('diff_cls_pair.csv')

#### class가 같은 이미지 pair중 하나를 제거 

In [78]:
remain = info_df[(~info_df.image_path.isin(set(res_df[res_df[3]==res_df[4]][1])))]

In [79]:
remain.to_csv('remain.csv')

#### value counts도 저장 

In [80]:
res2 = remain['class_name'].value_counts()

In [81]:
res2.to_csv('value_counts.csv', header=True)

#### 다른 방식으로 필터링된 CSV 파일을 불러와, 코사인 유사도를 사용하여 필터링된 이미지들을 반영한다.

In [82]:
cv_df = pd.read_csv('./train_reduce.csv')

In [83]:
cv_df['image_path'] = cv_df['image_path'].apply(lambda x: '/'.join(x.split('/')[3:]))

In [87]:
res = remain[remain['image_path'].isin(set(cv_df['image_path']))]

In [89]:
res.to_csv('del_dup_train.csv')

In [91]:
res['class_name'].value_counts()

class_name
n02389026    31
n01882714    31
n04606251    31
n02415577    31
n02088094    31
             ..
n01774384     4
n02493509     3
n02089973     2
n01740131     1
n02113712     1
Name: count, Length: 500, dtype: int64

In [138]:
import shutil
import os


#### 다른 class 유사도 높은 샘플 보기/

In [176]:
check = res_df[res_df[3]!=res_df[4]]

In [178]:
import os
import cv2
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import clear_output

# 데이터 준비 (이 부분은 실제 코드에 맞게 조정해야 합니다)
# miss_df, val_data_dir, class_name_to_idx 등의 데이터와 디렉토리 설정이 필요합니다

def plot_image(value):
    value = int(value)  # 슬라이더 값 추출
    clear_output(wait=True)  # 이전 출력 제거
    fig, ax = plt.subplots(1, 2, figsize=(6, 12))  # 새 그림 생성

    check_img = check.iloc[value]
    # 잘못 예측된 sample (miss_df에서 잘못 예측된 이미지 경로)
    img_path = os.path.join(train_root, check_img[0])
    image = cv2.imread(img_path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    ax[0].imshow(image)
    x1_label = check_img[3]
    ax[0].set_title(f'{x1_label}')
    ax[0].axis('off')
    

    img_path = os.path.join(train_root, check_img[1])
    image = cv2.imread(img_path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    ax[1].imshow(image)
    x2_label = check_img[4]
    ax[1].set_title(f'{x2_label}')
    ax[1].axis('off')


    plt.tight_layout()
    plt.show()

# 슬라이더 생성
slider = widgets.IntSlider(min=0, max=len(check) - 1, step=1, description='Index')

# 슬라이더와 plot_image 함수 연결
widgets.interact(plot_image, value=slider)


interactive(children=(IntSlider(value=0, description='Index', max=172), Output()), _dom_classes=('widget-inter…

<function __main__.plot_image(value)>

In [162]:
len(check)

173